## MODELLING - GPG - Government Equalities Office

### Target: "DiffMedianHourlyPercent"

#### 1 - General Preparation
- Unnecessary Columns
- Missing Values
- Imputing Values

#### 2 Feature Engineering
- Quantizise Company Size
- Include Company Sector

### Linear Model
Feature Engineering:


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path = "data/gender-pay-gap-uk-gov/2017.csv.gz"
df = pd.read_csv(path, compression='gzip')
rows = df.shape[0]
cols = df.shape[1]
print(f"Rows: {rows}")
print(f"Cols: {cols}")

Rows: 10562
Cols: 25


### 1 - General Preparation

In [3]:
df.head(3)

,EmployerName,Address,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,...,FemaleUpperMiddleQuartile,MaleTopQuartile,FemaleTopQuartile,CompanyLinkToGPGInfo,ResponsiblePerson,EmployerSize,CurrentName,SubmittedAfterTheDeadline,DueDate,DateSubmitted
0,"""Bryanston School"",Incorporated","Bryanston House,\r\nBlandford,\r\nDorset,\r\nD...",00226143,85310,18.0,28.2,0.0,0.0,0.0,0.0,...,50.8,51.5,48.5,https://www.bryanston.co.uk/employment,Nick McRobb (Bursar and Clerk to the Governors),500 to 999,BRYANSTON SCHOOL INCORPORATED,False,05/04/2018 00:00:00,27/03/2018 11:42:49
1,"""RED BAND"" CHEMICAL COMPANY, LIMITED","19 Smith's Place,\r\nLeith Walk,\r\nEdinburgh,...",SC016876,47730,2.3,-2.7,15.0,37.5,15.6,66.7,...,89.7,18.1,81.9,NaN,Philip Galt (Managing Director),250 to 499,"""RED BAND"" CHEMICAL COMPANY, LIMITED",False,05/04/2018 00:00:00,28/03/2018 16:44:25
2,118 LIMITED,"3 Alexandra Gate Ffordd Pengam,\r\nGround Floo...",03951948,61900,1.7,2.8,13.1,13.6,70.0,57.0,...,50.0,58.0,42.0,NaN,"Emma Crowe (VP, Human Resources)",500 to 999,118 LIMITED,False,05/04/2018 00:00:00,27/03/2018 19:10:41


In [4]:
print(df.nunique())

EmployerName                 10561
Address                       9039
CompanyNumber                 9203
SicCodes                      1943
DiffMeanHourlyPercent          829
DiffMedianHourlyPercent        891
DiffMeanBonusPercent          1763
DiffMedianBonusPercent        1734
MaleBonusPercent               978
FemaleBonusPercent             979
MaleLowerQuartile              980
FemaleLowerQuartile            980
MaleLowerMiddleQuartile        986
FemaleLowerMiddleQuartile      986
MaleUpperMiddleQuartile        974
FemaleUpperMiddleQuartile      974
MaleTopQuartile                944
FemaleTopQuartile              944
CompanyLinkToGPGInfo          6589
ResponsiblePerson             7304
EmployerSize                     7
CurrentName                  10561
SubmittedAfterTheDeadline        2
DueDate                          2
DateSubmitted                10474
dtype: int64


In [5]:
df['DueDate'].value_counts()
df['CompanyLinkToGPGInfo'].sample(10)

4297                                                   NaN
7210       https://www.prs.uk.com/gender-pay-gap-reporting
9512                                 http://www.tbgs.co.uk
6344     https://www.vipelectroniccigarette.co.uk/about...
3557                             http://www.g1group.co.uk/
5104                     http://www.rowlandspharmacy.co.uk
814      https://www.balfourbeatty.com/investors/result...
5739               https://www.mbna.co.uk/news/gender-pay/
8210     https://www.playstation.com/uk-gender-pay-gap-...
10080    http://www.ward-security.co.uk/ward-security-g...
Name: CompanyLinkToGPGInfo, dtype: object

### 2 - Feature Cleaning
**Dropping Data:**
- EmployerName / CurrentName: are unique in every row
- CompanyLinkToGPGInfo: contains urls
- ResponsiblePerson: Maybe we could check whether the companies that didn't fill this field also skipped PDFs or if they're incomplete or have a pattern.
- DueDate / DateSubmitted / SubmittedAfterTheDeadline 

In [6]:
columns_to_drop = [
    'EmployerName','CurrentName','CompanyNumber',
    'CompanyLinkToGPGInfo','ResponsiblePerson',
    'DueDate','DateSubmitted','SubmittedAfterTheDeadline'
]
df_cols = df.drop(columns_to_drop, axis='columns')
df.shape

(10562, 25)

#### Imputing values

In [7]:
def check_presence(df):
    return df.notnull().sum() / rows
check_presence(df_cols)

Address                      0.974153
SicCodes                     0.946317
DiffMeanHourlyPercent        1.000000
DiffMedianHourlyPercent      1.000000
DiffMeanBonusPercent         0.998106
DiffMedianBonusPercent       0.998106
MaleBonusPercent             1.000000
FemaleBonusPercent           1.000000
MaleLowerQuartile            1.000000
FemaleLowerQuartile          1.000000
MaleLowerMiddleQuartile      1.000000
FemaleLowerMiddleQuartile    1.000000
MaleUpperMiddleQuartile      1.000000
FemaleUpperMiddleQuartile    1.000000
MaleTopQuartile              1.000000
FemaleTopQuartile            1.000000
EmployerSize                 1.000000
dtype: float64

In [8]:
# Mean because the measurement is mean
mean_bonus_percent = df_cols['DiffMeanBonusPercent'].mean()
df_cols['DiffMeanBonusPercent'] = df_cols['DiffMeanBonusPercent'].fillna(mean_bonus_percent)

# Median because the measurement is median
median_bonus_percent = df['DiffMedianBonusPercent'].median()
df_cols['DiffMedianBonusPercent'] = df_cols['DiffMedianBonusPercent'].fillna(median_bonus_percent)

# Mode because it is categorical
employer_size_mode = df_cols['EmployerSize'].mode()
df_cols['EmployerSize'] = df_cols['EmployerSize'].fillna(employer_size_mode)

# employer_size_mode
check_presence(df_cols)

Address                      0.974153
SicCodes                     0.946317
DiffMeanHourlyPercent        1.000000
DiffMedianHourlyPercent      1.000000
DiffMeanBonusPercent         1.000000
DiffMedianBonusPercent       1.000000
MaleBonusPercent             1.000000
FemaleBonusPercent           1.000000
MaleLowerQuartile            1.000000
FemaleLowerQuartile          1.000000
MaleLowerMiddleQuartile      1.000000
FemaleLowerMiddleQuartile    1.000000
MaleUpperMiddleQuartile      1.000000
FemaleUpperMiddleQuartile    1.000000
MaleTopQuartile              1.000000
FemaleTopQuartile            1.000000
EmployerSize                 1.000000
dtype: float64

In [9]:
df_cols['EmployerSize'].replace('Not Provided', '250 to 499', inplace=True)
df_cols['EmployerSize'].value_counts()
# check_presence(df)

250 to 499        5026
500 to 999        2534
1000 to 4999      2208
5000 to 19,999     441
Less than 250      288
20,000 or more      65
Name: EmployerSize, dtype: int64

#### EmpoyerSize range to quantitative category

In [21]:
# Could come a copy here.
cat_to_quant = {
 'Less than 250' : 125,
 '250 to 499' : 375,
 '500 to 999': 750,
 '1000 to 4999' : 2500,
 '5000 to 19,999': 15000,
 '20,000 or more': 35000
}

# DO NOT DO IT TWICE
# df_cols['EmployerSize'] = df_cols['EmployerSize'].map(cat_to_quant)

df_cols['EmployerSize'].value_counts()
# check_presence(df)

375      5026
750      2534
2500     2208
15000     441
125       288
35000      65
Name: EmployerSize, dtype: int64

### 3 Linear Model

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

features = [
    'DiffMeanBonusPercent', 'DiffMedianBonusPercent',
    'MaleBonusPercent', 'FemaleBonusPercent',
    'MaleLowerQuartile', 'FemaleLowerQuartile',
    'MaleLowerMiddleQuartile', 'FemaleLowerMiddleQuartile',
    'MaleUpperMiddleQuartile', 'FemaleUpperMiddleQuartile',
    'MaleTopQuartile', 'FemaleTopQuartile', 'EmployerSize'
]

In [17]:
X = df_cols[features]
y = df_cols['DiffMedianHourlyPercent']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(7921, 13) (7921,)
(2641, 13) (2641,)


In [18]:
linear_reg = LinearRegression(normalize=True)
linear_reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [19]:
y_preds_test = linear_reg.predict(X_test)
y_preds_train = linear_reg.predict(X_train)

train_score = r2_score(y_train, y_preds_train, multioutput='raw_values')
test_score = r2_score(y_test, y_preds_test, multioutput='raw_values')

print("Train Score:", train_score)
print("Test Score:", test_score)

Train Score: [0.49322808]
Test Score: [0.5006404]


#### Adding Company Sector

In [27]:
sic_codes = pd.read_csv("data/siccodes/uk-sic-2007-condensed_csv.csv")
sic_codes.shape

(731, 5)

In [28]:
sic_codes.head(3)

,sic_code,sic_description,section,section_description,sic_version
0,1110,"Growing of cereals (except rice), leguminous c...",A,"Agriculture, Forestry and Fishing",SIC 2007
1,1120,Growing of rice,A,"Agriculture, Forestry and Fishing",SIC 2007
2,1130,"Growing of vegetables and melons, roots and tu...",A,"Agriculture, Forestry and Fishing",SIC 2007


In [54]:
sic_codes[sic_codes.sic_code == 1110]

,sic_code,sic_description,section,section_description,sic_version
0,1110,"Growing of cereals (except rice), leguminous c...",A,"Agriculture, Forestry and Fishing",SIC 2007


In [32]:
sic_codes.groupby("section_description").count()

21

In [37]:
df_cols['SicCodes'].sample(3)

3808    17219
3786    46460
2557        1
Name: SicCodes, dtype: object

In [49]:
# df_cols['SicCodes'] = pd.to_numeric(df_cols['SicCodes'])
# df_cols.shape
df_cols['SicCodes'].value_counts()

82990                                  446
1                                      336
1,\r\n84110                            306
70100                                  277
78200                                  209
                                      ... 
86101,\r\n86102                          1
87300,\r\n88100,\r\n88910,\r\n88990      1
61300                                    1
1,\r\n49390                              1
85590,\r\n90040,\r\n91011,\r\n93110      1
Name: SicCodes, Length: 1943, dtype: int64